# OpenPose Jupyter Data Exploration
## Imports and dependencies

In [ ]:
import pandas as pd
from pandas import DataFrame

import plotly.express as px
import plotly.graph_objects as go
from glob import glob

from path import Path

Mandatory input:
* frame data path
* frame files count
* persons in frame count

## Import data

In [2]:
def get_frame_data(frame_file):
    try:
        frame_data = pd.read_csv(frame_file)
        
        return frame_data
    except FileNotFoundError:
        print(f"Frame data not found for frame {frame_idx}")  
        return pd.DataFrame()

In [3]:
def get_person_files(input_files, frame_data_path):
    all_persons_files = {}
    max_persons = 10
    
    for input_file_idx in range(len(input_files)):
        for person_idx in range(max_persons):
            expected_file = frame_data_path.replace("[frame_idx]", str(input_file_idx))
            expected_file = expected_file.replace("[person_idx]", str(person_idx))
            
            if expected_file in input_files:
                person_files = []
                
                if person_idx in all_persons_files:
                    person_files = all_persons_files[person_idx]
                
                person_files.append(expected_file)
                
                all_persons_files[person_idx] = person_files
                
    persons_indeces = all_persons_files.keys()
    print(f"Found {len(persons_indeces)} persons")
    for person_idx in persons_indeces:
        print(f"Person {person_idx} has {len(all_persons_files[person_idx])} frame files")
    
    return all_persons_files      

In [4]:
frames_root_path = "/Users/allarviinamae/EduWorkspace/master-thesis-training-videos/output/backflip-1-allar"
frame_data_path = f"{frames_root_path}/backflip-1-allar.mov-[frame_idx]-[person_idx].csv"

print(f"Frame data path={frame_data_path}")
        
input_files = [Path(f).abspath() for f in glob(frames_root_path + '/*')]

input_files_count = len(input_files)
print(f"Root path includes {input_files_count} files")

all_persons_files = get_person_files(input_files, frame_data_path)

# Collecting frame data for the person with index 0
person_idx_to_collect = 0
person_frame_files = all_persons_files[person_idx_to_collect]

frame_data = [get_frame_data(frame_file) for frame_file in person_frame_files]
 
frame_count = len(frame_data)
print(f"Imported data for {frame_count} frames")

Frame data path=/Users/allarviinamae/EduWorkspace/master-thesis-training-videos/output/backflip-1-allar/backflip-1-allar.mov-[frame_idx]-[person_idx].csv
Root path includes 87 files
Found 3 persons
Person 0 has 83 frame files
Person 1 has 3 frame files
Person 2 has 1 frame files
Imported data for 83 frames


## Define some functions

In [5]:
def get_min_recognized_coord(df):
    min_recognized = 9999
    
    for coord in df.iloc[:, 0]:
        if coord < min_recognized and coord != 0:
            min_recognized = coord
            
    return min_recognized

def get_xmin_xmax(df):
    x_min = get_min_recognized_coord(df)
    x_max = max(df.iloc[:, 0])
    
    return (x_min, x_max)

def get_minmax_avg(minmax):
    return (minmax[0] + minmax[1]) / 2

In [35]:
def get_body_part(body_connection, df_orig):
    frm = body_connection[0]
    to = body_connection[1]
    
    df = df_orig.copy()
    # df['y'] *= -1
    
    if df.iloc[to, 0] == 0 or df.iloc[to, 1] == 0:
        return
    if df.iloc[frm, 0] == 0 or df.iloc[frm, 1] == 0:
        return
    
    x0 = df.iloc[frm, 0]
    y0 = df.iloc[frm, 1]
    
    x1 = df.iloc[to, 0]
    y1 = df.iloc[to, 1]
    
    return go.Scatter(x=[x0, x1], y=[y0, y1], name=body_connection[2])

def get_all_body_parts(df):
    body_connections = [(0, 1, 'Neck'),
                        (15, 0, 'REye'),
                        (16, 0, 'LEye'),
                        (1, 8, 'Spine'),
                        (1, 2, 'RShoulder'),
                        (2, 3, 'RArm'),
                        (3, 4, 'RForearm'),
                        (1, 5, 'LShoulder'),
                        (5, 6, 'LArm'),
                        (6, 7, 'LForearm'),
                        (8, 9, 'RHip'),
                        (9, 10, 'RThigh'),
                        (10, 11, 'RCalf'),
                        (11, 22, 'RFoot'),
                        (8, 12, 'LHip'),
                        (12, 13, 'LThigh'),
                        (13, 14, 'LCalf'),
                        (14, 19, 'LFoot')]
    
    return [get_body_part(body_connection, df) for body_connection in body_connections if get_body_part(body_connection, df) != None]  

def get_frames(df):
    return [go.Frame(data=get_all_body_parts(df[i])) for i in range(1, frame_count)]

def add_body_part(frm, to, df, fig):
    if df.iloc[to, 0] == 0 or df.iloc[to, 1] == 0:
        return
    if df.iloc[frm, 0] == 0 or df.iloc[frm, 1] == 0:
        return
    
    fig.add_shape(
        go.layout.Shape(
            type="line",
            x0=df.iloc[frm, 0],
            y0=df.iloc[frm, 1],
            x1=df.iloc[to, 0],
            y1=df.iloc[to, 1],
            line=dict(
                color="RoyalBlue",
            ),
        ))
    
# Body Parts
def add_body_parts(df_orig, fig):
    df = df_orig.copy()
    #df['y'] *= -1
    
    add_body_part(0, 1, df, fig)
    add_body_part(1, 8, df, fig)

    add_body_part(1, 2, df, fig)
    add_body_part(2, 3, df, fig)
    add_body_part(3, 4, df, fig)

    add_body_part(1, 5, df, fig)
    add_body_part(5, 6, df, fig)
    add_body_part(6, 7, df, fig)

    add_body_part(8, 9, df, fig)
    add_body_part(9, 10, df, fig)
    add_body_part(10, 11, df, fig)

    add_body_part(8, 12, df, fig)
    add_body_part(12, 13, df, fig)
    add_body_part(13, 14, df, fig)
    
def add_all_body_parts():
    skips = 5
    skip = skips
    
    for i, frame in enumerate(frame_data):
        if skip > 0 and i != 1 and i != 60:
            skip = skip - 1
            continue
        else:
            skip = skips
    
        print(f"Adding frame {i}")
        
        add_body_parts(frame, fig)

In [44]:
def render_frame(frame):
    fig = go.Figure()
    fig.update_xaxes(range=[-1500, 2500])
    fig.update_yaxes(range=[-1500, 1500])
    fig.update_yaxes(autorange="reversed")
        
    add_body_parts(frame, fig)

    fig.show()
        
render_frame(frame_data[5])

In [45]:
def get_frame_wo_x_zeros(frame):
    return frame[frame['x'] != 0]

def get_frame_wo_y_zeros(frame):
    return frame[frame['y'] != 0]

def get_max_x_coord(frame_data):
    return max([max(frame.iloc[:, 0]) for frame in frame_data])

def get_min_x_coord(frame_data):    
    return min([min(get_frame_wo_x_zeros(frame).iloc[:, 0]) for frame in frame_data])

def get_max_y_coord(frame_data):
    return max([max(frame.iloc[:, 1]) for frame in frame_data])

def get_min_y_coord(frame_data):    
    return min([min(get_frame_wo_y_zeros(frame).iloc[:, 0]) for frame in frame_data])

max_x_coord = get_max_x_coord(frame_data)
min_x_coord = get_min_x_coord(frame_data)

max_y_coord = get_max_y_coord(frame_data)
min_y_coord = get_min_y_coord(frame_data)

print(f"x max={max_x_coord}, x min={min_x_coord}, y max={max_y_coord}, y min={min_y_coord}")

x max=1298.431640625, x min=854.0137329101561, y max=971.708740234375, y min=854.0137329101561


In [49]:
def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

def get_animated_figure(frame_data, max_x_coord, max_y_coord):
    fig = go.Figure(
        data = get_all_body_parts(frame_data[0]),
        layout = go.Layout(
            xaxis=dict(range=[0, max_x_coord + 200], autorange=False),
            yaxis=dict(range=[0, max_y_coord + 200], autorange=False),
            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None, frame_args(50)]
                             ),
                         dict(label="Pause",
                              method="animate",
                              args=[[None], frame_args(0)]
                             )
                        ]
            )]
        ),
        frames=get_frames(frame_data)
    )
    
    return fig

fig = get_animated_figure(frame_data, max_x_coord, max_y_coord)
fig.show()

In [ ]:
def get_body_part_y_data(frame, body_part_nr=0):
    return float(frame.iloc[body_part_nr, 1:2])

def get_body_part_x_data(frame, body_part_nr=0):
    return float(frame.iloc[body_part_nr, 0:1])

def show_body_part_trajectory(body_part_nr=0, coord="Y"):
    if coord == "Y":
        body_part_data = [[idx, get_body_part_y_data(frame, body_part_nr)] for idx, frame in enumerate(frame_data)]
    elif coord == "X":
        body_part_data = [[idx, get_body_part_x_data(frame, body_part_nr)] for idx, frame in enumerate(frame_data)]

    df = pd.DataFrame(body_part_data, columns = ['Frames', 'YValues']) 

    fig = px.line(df, x='Frames', y='YValues')
    fig.show()

# 0 = nose
# 8 = mid hip
show_body_part_trajectory(body_part_nr=0, coord="Y")

In [ ]:
fixed_frame_data = [frame.copy() for frame in frame_data]

def get_new_body_part_data(body_part_data):
    new_body_part_data = []
    
    for idx, body_part in enumerate(body_part_data):
        if idx == 0:
            new_body_part_data.append(body_part_data[0])
            continue
            
        previous_body_part = body_part_data[idx - 1]
        
        #if abs(body_part - previous_body_part) > 100:
        #    new_body_part_data.append(previous_body_part)
        #else:
        new_body_part_data.append(body_part)
            
    return new_body_part_data

def fix_body_part_data(frame_data, body_part_nr=0):    
    body_part_y_data = [get_body_part_y_data(frame, body_part_nr) for idx, frame in enumerate(frame_data)]
    body_part_x_data = [get_body_part_y_data(frame, body_part_nr) for idx, frame in enumerate(frame_data)]
    
    new_body_part_y_data = get_new_body_part_data(body_part_y_data)
    new_body_part_x_data = get_new_body_part_data(body_part_x_data)
    
    for i, _ in enumerate(body_part_x_data):
        if body_part_x_data[i] == new_body_part_x_data[i]:
            print(body_part_x_data[i])
            print(new_body_part_x_data[i])
      
    for idx, frame in enumerate(frame_data):
        frame.iloc[body_part_nr, 1:2] = new_body_part_y_data[idx]
        frame.iloc[body_part_nr, 0:1] = new_body_part_x_data[idx]
        
        frame_data[idx] = frame
        
    return frame_data
    
fixed_frame_data = fix_body_part_data(fixed_frame_data, 0)

In [ ]:
fig = go.Figure(
    data = get_all_body_parts(fixed_frame_data[0]),
    layout = go.Layout(
        xaxis=dict(range=[0, max_x_coord + 200], autorange=False),
        yaxis=dict(range=[-(max_y_coord + 200), 0], autorange=False),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, frame_args(50)]
                         ),
                     dict(label="Pause",
                          method="animate",
                          args=[[None], frame_args(0)]
                         )
                    ]
        )]
    ),
    frames=get_frames(fixed_frame_data)
)

fig.show()